## Assemble a DS

In [62]:
import pandas as pd

art = pd.read_csv(
    "../data/processed/articles_clean.csv",
    usecols=["sku", "groupId", "name", "name.1", "status", "audience", "category", "priceSEK", "description", "color"],
    dtype=str
)

In [63]:
#proportion of price in art?
art['priceSEK'].isna().mean()

0.14844555016106123

In [64]:
# For rows in art where priceSEK is null, try to fill from transactions_clean.csv

# Load transactions with only sku and price_sek
trans = pd.read_csv(
    "../data/processed/transactions_clean.csv",
    usecols=["sku", "price_sek"],
    dtype=str
)

# Convert price_sek to numeric, coerce errors to NaN
trans['price_sek'] = pd.to_numeric(trans['price_sek'], errors='coerce')
# Set all prices below 1 to NaN
trans.loc[trans['price_sek'] < 1, 'price_sek'] = pd.NA

# For each sku, get the median price from transactions
sku_price = trans.groupby('sku', dropna=False)['price_sek'].median()

# Map sku in art to median price from transactions for missing priceSEK
mask = art['priceSEK'].isna()
art.loc[mask, 'priceSEK'] = art.loc[mask, 'sku'].map(sku_price)


In [65]:
# Convert 'price' to numeric, coerce errors to NaN
art['priceSEK'] = pd.to_numeric(art['priceSEK'], errors='coerce')
# Set all prices below 1 to NaN
art.loc[art['priceSEK'] < 1, 'priceSEK'] = pd.NA


In [66]:
#proportion of price in art?
art['priceSEK'].isna().mean()

0.06771998551841296

In [67]:
art['priceSEK'].describe()

count    100428.000000
mean        468.930444
std         351.603877
min           1.000000
25%         318.000000
50%         439.000000
75%         579.000000
max       34998.000000
Name: priceSEK, dtype: float64

In [68]:
# Create price buckets based on the distribution of price_sek, using 6 buckets
price_bins = [0, 100, 300, 600, 1000, 2000, float('inf')]
price_labels = [
    'Budget',        # 0-100
    'Value',         # 100-300
    'Popular',       # 300-600
    'Premium',       # 600-1000
    'Luxury',        # 1000-2000
    'Exclusive'      # 2000+
]
art['priceband'] = pd.cut(art['priceSEK'], bins=price_bins, labels=price_labels, include_lowest=True)
art['priceband'] = art['priceband'].astype(object)
art.loc[art['priceSEK'].isna(), 'priceband'] = 'unknown'

In [69]:
if 'description' in art.columns:
    art['description'] = art['description'].fillna('unknown')
art = art[art["priceband"] != "unknown"].drop(columns=["priceSEK", "status"])

In [70]:
art

,sku,groupId,name,name.1,color,audience,category,description,priceband
0,052743,052743,Lakan örngott,unknown,blå,unknown,unknown,unknown,Value
1,055522,055522,Tröja,Gjestal Garn,NaN,dam,Tröjor,Sticka en färgglad och trendig tröja i garnet Vera från House of Yarn!,Budget
2,055573,055573,Luva,Novita,vit,dam,"Mössor & hattar,Mönster",Sticka en trendig huva i garnet Halaus från Novita!,Budget
3,055575,055575,Vantar,Novita,vit,dam,Vantar,Sticka ett par vantar med blomstermotiv i garnet Halaus från Novita!,Budget
4,055576,055576,Benvärmare,Novita,vit,dam,Sockor & strumpor,Sticka ett par trendiga benvärmare i garnet Halaus från Novita!,Budget
...,...,...,...,...,...,...,...,...,...
107618,AH3021-4244,AH3021,Stödstrumpa Herr,Funq Wear,blå,generic,"Stödstrumpor,Stödartiklar","FUNQ WEAR Komfortsockor MILD 11-14 mmHg Organic Cotton är graderade kompressionssockor av högsta kvalitet. Dessa är utprovade på både kvinnor och män. Det extra stödet kring ankeln ökar blodcirkulationen och motverkar svullnad i fötterna. FUNQ WEAR Komfortsockor MILD Organic Cotton uppskattas för sin färgglada design av bla resande, stillasittande, gravida samt för dig som står och går mycket på jobbet. De är lämpliga för vardagligt bruk och perfekta för varmare dagar eller för dig som föredrar ankelsockor framför knästrumpor. Komfortsockorna stickas av ekologisk bomull av högsta kvalitet. Kan krympa något efter tvätt. 55% EKO bomull, 30% polyamid och 15% lycra. Tvätt 60°. Färg blå.",Value
107619,AH3021-4547,AH3021,Stödstrumpa Herr,Funq Wear,blå,generic,"Stödstrumpor,Stödartiklar","FUNQ WEAR Komfortsockor MILD 11-14 mmHg Organic Cotton är graderade kompressionssockor av högsta kvalitet. Dessa är utprovade på både kvinnor och män. Det extra stödet kring ankeln ökar blodcirkulationen och motverkar svullnad i fötterna. FUNQ WEAR Komfortsockor MILD Organic Cotton uppskattas för sin färgglada design av bla resande, stillasittande, gravida samt för dig som står och går mycket på jobbet. De är lämpliga för vardagligt bruk och perfekta för varmare dagar eller för dig som föredrar ankelsockor framför knästrumpor. Komfortsockorna stickas av ekologisk bomull av högsta kvalitet. Kan krympa något efter tvätt. 55% EKO bomull, 30% polyamid och 15% lycra. Tvätt 60°. Färg blå.",Value
107713,SOFIND,SOFIND,Bruksanvisng Sofia N DK,unknown,NaN,unknown,unknown,unknown,Budget
107715,TEST01,TEST01,guldarmband,Disney,guld,unknown,Smycken,Fint guldarmband,Popular


## only one group_id should remain

In [71]:
# Deduplicate so only one row per groupId remains, keeping the one with the most information (fewest NA/unknown), and drop sku
def info_score(row):
    # Count number of non-missing, non-unknown fields (excluding sku and groupId)
    fields = [col for col in art.columns if col not in ("sku", "groupId")]
    score = 0
    for col in fields:
        val = row[col]
        if pd.isna(val):
            continue
        sval = str(val).strip().lower()
        if sval in {"", "unknown", "nan", "none"}:
            continue
        score += 1
    return score

art = art.assign(_info_score=art.apply(info_score, axis=1))
art = art.sort_values("_info_score", ascending=False)
art = art.drop_duplicates(subset=["groupId"], keep="first").drop(columns=["_info_score", "sku"]).reset_index(drop=True)


In [72]:
art

,groupId,name,name.1,color,audience,category,description,priceband
0,261290,Bygel bh Shine,Miss Mary,champagne,dam,"Bygel-bh,Bh,Underkläder","Shine broderad bygel-bh från Miss Mary i en feminin modell i glansig charmeuse med brodyrdetaljer. Klassisk trepartskärning på kupan ger optimal rund passform. Den ovadderade kupan är tillverkad i stum charmeuse vilket håller bysten stadigt på plats.Upphöjd nederkant framtill ger ökad komfort vid sittande läge ochhindrar att bandet viker sig. Skärningen låter bygeln komma närmre kroppen så att den kan ge optimalt stöd och separera bysten. Vadderade, reglerbara komfortaxelband som avlastar axlarna. Elastisk u-formad rygg med justerbar hak- hyskknäppning.Matcha gärna med trosa i samma serie art.no",Popular
1,264440,Sport bh,Glamorise,rosa,dam,"Sport-bh,Bh utan bygel,Bh,Underkläder","Sport-bh från Glamorise med justerbar kontrollerande framdel. Axelbanden anpassar du i 4 olika nivåer för låg, medium eller hårt träningspass. Sömlösa innerkupor av snabbtorkande material transporterar bort fukten. Unik korsdesign utan byglar, separerar och håller in bysten. Ställbara, breda axelband. 56% polyester, 37% polyamid, 7% elastan. Fintvätt 40°.",Popular
2,260313,Bh utan bygel med Magic Lift funktion,Glamorise,rosa,dam,"Bh utan bygel,Bh,Underkläder","En underbar bh från Glamorise i ett andasmaterial som transportera bort fukten och håller dig fräsch hela dagen. Den unika Magic-Lift funktionen ger stöd och support och formar din byst på ett vackert sätt. Breda reglerbara axelband. 51% polyamide, 37% polyester, 12% elastan. Fintvätt 40.",Popular
3,545906,Bh utan bygel Clara,Anita,champagne,dam,"Bh utan bygel,Bh,Underkläder","Clara- en stilren från Anita, vacker bh sydd som trepart vilket ger mycket bra stöd och håller in bysten fint vid sidan- detta gör att modellen finns ända upp till H-kupa . Eftersom övre delen av kupan är transparant ger detta en minimizer-effekt, då bysten ser nättare ut- fantastisk passform upp till st 120! Sköna vadderade justerbara axelband som skonar axlarna. Ryggparti samt axelband blir proportionerligt större vid större storlekar. 85% nylon, 15% elastan. Fintvätt 40°.",Premium
4,266064,Framknäppt Bygel bh,Glamorise,grön,dam,"Bygel-bh,Framknäppt bh,Bh,Underkläder","Framknäppt wonderwire-bh från Glamorise. Gilla ditt utseende och må bra. Komplett med en smickrande urringning, vackra spetsöverkupor och bekväm knäppning. Silkeslena fuktavledande underkupor ger enastående komfort. Utjämnande formande delar på kupornas sidor ger en smickarnde silhuett. Breda vadderade axelband garanterar komfort hela dagen. Det dolda bekväma resåret följer bystens kontur så att bygeln aldrig kommer i kontakt med kroppen. 67% polyamid, 26% polyester, 7% elastan. Handtvätt 30°.",Popular
...,...,...,...,...,...,...,...,...
8357,202986,Väska tetrimönstrad,unknown,NaN,unknown,unknown,unknown,Value
8358,202960,Fleeceset,unknown,NaN,unknown,unknown,unknown,Value
8359,361065,Tupp & höna,unknown,NaN,unknown,unknown,unknown,Budget
8360,270348,kappa,unknown,NaN,unknown,unknown,unknown,Popular


## 1. Build a clean text field for vectorization

In [73]:
import pandas as pd, unicodedata, re

MISSING = {"", "unknown", "nan", "none"}

def clean(x):
    if pd.isna(x): return ""
    s = str(x).strip()
    return "" if s.lower() in MISSING else s

def canon(s):
    s = unicodedata.normalize("NFKC", s)
    s = s.replace("&", " ").replace(",", " ")
    s = re.sub(r"\u00A0", " ", s)
    s = re.sub(r"[\u2010-\u2015\u2212\-]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def to_token(s):
    return re.sub(r"\s+", "_", s.strip())

def tidy_underscores(s):
    s = re.sub(r"\s*_\s*", "_", s)
    return re.sub(r"_+", "_", s)

def norm_category(cat):
    if pd.isna(cat): return ""
    return " ".join(to_token(clean(p)) for p in str(cat).split(",") if clean(p))

def build_text(row):
    names = [to_token(clean(row.get(c, ""))) for c in ("name", "name.1") if clean(row.get(c, ""))]
    others = [
        clean(row.get("color", "")),
        clean(row.get("audience", "")),
        norm_category(row.get("category", "")),
        clean(row.get("priceband", "")),
        clean(row.get("description", "")),
    ]
    s = " ".join(names*3 + [o for o in others if o])
    return tidy_underscores(canon(s))

art["text"] = art.apply(build_text, axis=1)
group_df = art.loc[art["text"].ne(""), ["groupId", "text"]].reset_index(drop=True)

In [74]:
pd.set_option('display.max_colwidth', None)
group_df.head()

,groupId,text
0,261290,Bygel_bh_Shine Miss_Mary Bygel_bh_Shine Miss_Mary Bygel_bh_Shine Miss_Mary champagne dam Bygel bh Bh Underkläder Popular Shine broderad bygel bh från Miss Mary i en feminin modell i glansig charmeuse med brodyrdetaljer. Klassisk trepartskärning på kupan ger optimal rund passform. Den ovadderade kupan är tillverkad i stum charmeuse vilket håller bysten stadigt på plats.Upphöjd nederkant framtill ger ökad komfort vid sittande läge ochhindrar att bandet viker sig. Skärningen låter bygeln komma närmre kroppen så att den kan ge optimalt stöd och separera bysten. Vadderade reglerbara komfortaxelband som avlastar axlarna. Elastisk u formad rygg med justerbar hak hyskknäppning.Matcha gärna med trosa i samma serie art.no
1,264440,Sport_bh Glamorise Sport_bh Glamorise Sport_bh Glamorise rosa dam Sport bh Bh_utan_bygel Bh Underkläder Popular Sport bh från Glamorise med justerbar kontrollerande framdel. Axelbanden anpassar du i 4 olika nivåer för låg medium eller hårt träningspass. Sömlösa innerkupor av snabbtorkande material transporterar bort fukten. Unik korsdesign utan byglar separerar och håller in bysten. Ställbara breda axelband. 56% polyester 37% polyamid 7% elastan. Fintvätt 40°.
2,260313,Bh_utan_bygel_med_Magic_Lift_funktion Glamorise Bh_utan_bygel_med_Magic_Lift_funktion Glamorise Bh_utan_bygel_med_Magic_Lift_funktion Glamorise rosa dam Bh_utan_bygel Bh Underkläder Popular En underbar bh från Glamorise i ett andasmaterial som transportera bort fukten och håller dig fräsch hela dagen. Den unika Magic Lift funktionen ger stöd och support och formar din byst på ett vackert sätt. Breda reglerbara axelband. 51% polyamide 37% polyester 12% elastan. Fintvätt 40.
3,545906,Bh_utan_bygel_Clara Anita Bh_utan_bygel_Clara Anita Bh_utan_bygel_Clara Anita champagne dam Bh_utan_bygel Bh Underkläder Premium Clara en stilren från Anita vacker bh sydd som trepart vilket ger mycket bra stöd och håller in bysten fint vid sidan detta gör att modellen finns ända upp till H kupa . Eftersom övre delen av kupan är transparant ger detta en minimizer effekt då bysten ser nättare ut fantastisk passform upp till st 120! Sköna vadderade justerbara axelband som skonar axlarna. Ryggparti samt axelband blir proportionerligt större vid större storlekar. 85% nylon 15% elastan. Fintvätt 40°.
4,266064,Framknäppt_Bygel_bh Glamorise Framknäppt_Bygel_bh Glamorise Framknäppt_Bygel_bh Glamorise grön dam Bygel bh Framknäppt_bh Bh Underkläder Popular Framknäppt wonderwire bh från Glamorise. Gilla ditt utseende och må bra. Komplett med en smickrande urringning vackra spetsöverkupor och bekväm knäppning. Silkeslena fuktavledande underkupor ger enastående komfort. Utjämnande formande delar på kupornas sidor ger en smickarnde silhuett. Breda vadderade axelband garanterar komfort hela dagen. Det dolda bekväma resåret följer bystens kontur så att bygeln aldrig kommer i kontakt med kroppen. 67% polyamid 26% polyester 7% elastan. Handtvätt 30°.


## 2. Vectorize with TF-IDF (Term Frequency × Inverse Document Frequency)

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    lowercase=True,
    ngram_range=(1, 2), #use unigrams + bigrams
    min_df=1,
    strip_accents=None,   # keep å/ä/ö
    # sublinear_tf=True,  # optional
    # dtype=np.float32,   # optional memory saver
    token_pattern=r'(?u)\b\w+\b'
)
X_tfidf = tfidf.fit_transform(group_df["text"])
X_tfidf.shape

(8362, 94099)

## 3. Singular Value Decomposition + L2 normalize

TF-IDF gives precise but sparse signals; SVD compresses & generalizes them.
L2-norm makes nearest-neighbor search stable and comparable across items.

In [15]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize

n_components = min(128, max(2, X_tfidf.shape[1]-1)) 
svd = TruncatedSVD(n_components=n_components, random_state=0)
X_svd = svd.fit_transform(X_tfidf)
X_emb = normalize(X_svd)
X_emb.shape


(8362, 128)

## 4. Build 10-nearest neighbors (cosine) and return a small recs table


* It’s the **cosine of the angle** between vectors $a$ and $b$:

  $$
  \text{cosine\_sim}(a,b)=\frac{a\cdot b}{\|a\|\;\|b\|}
  $$

  * $=1$ → same direction (very similar)
  * $=0$ → orthogonal (unrelated)
  * $=-1$ → opposite (rare with TF-IDF since values are ≥0)

* After we **L2-normalize** vectors, cosine similarity becomes just the **dot product**.

* In scikit-learn, `metric="cosine"` actually computes **cosine distance**:

  $$
  \text{cosine\_dist} = 1 - \text{cosine\_sim}
  $$

  That’s why in the code we convert back with `similarity = 1 - d`.

Why we use it: it’s **scale-invariant** (ignores length), so two SKUs with similar wording but different text lengths still match well.


In [16]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

POOL = min(11, len(group_df))
nn = NearestNeighbors(metric="cosine", n_neighbors=POOL).fit(X_emb)
dists, idxs = nn.kneighbors(X_emb)

groupid_arr = group_df["groupId"].values

# Build all recommendations, then filter out self-recommendations
src_groupids = np.repeat(groupid_arr, POOL)
rec_groupids = groupid_arr[idxs.ravel()]
similarities = 1 - dists.ravel()

recs = pd.DataFrame({
    "src_groupId": src_groupids,
    "rec_groupId": rec_groupids,
    "similarity": similarities
})

# Remove self-recommendations
recs = recs[recs["src_groupId"] != recs["rec_groupId"]].copy()

# For each src_groupId, keep only the top POOL-1 recommendations (in case of ties or accidental duplicates)
recs["rec_rank"] = (
    recs.groupby("src_groupId")["similarity"]
    .rank(method="first", ascending=False)
    .astype(int)
)
recs = recs[recs["rec_rank"] <= POOL-1]

recs.head(10)

,src_groupId,rec_groupId,similarity,rec_rank
1,261290,261375,0.972898,1
2,261290,261296,0.969619,2
3,261290,261002,0.958648,3
4,261290,261371,0.957110,4
5,261290,261649,0.953067,5
6,261290,266882,0.952363,6
7,261290,261221,0.948965,7
8,261290,261561,0.947565,8
9,261290,261228,0.943912,9
10,261290,261564,0.934360,10


In [17]:
recs.sample(15, random_state=42)

,src_groupId,rec_groupId,similarity,rec_rank
60124,213967,218279,0.864835,9
36425,210317,210320,0.815785,4
77632,270272,271601,0.980396,5
63687,204107,205591,0.866523,8
6807,263897,263608,0.933692,9
24499,400271,400274,0.999918,2
10210,262303,262311,0.953345,2
41346,269402,268003,0.922150,8
76359,538959,518076,0.980136,8
50860,500561,529461,0.849313,7


In [18]:
target = "190041"  # example groupId

# pick columns and collapse art to one row per groupId
keep = [c for c in ["groupId","name","name.1","color","audience","category","priceband","description"] if c in art.columns]
details = art.drop_duplicates("groupId")[keep].copy()

view = (
    recs.loc[recs["src_groupId"].eq(target), ["rec_groupId","rec_rank","similarity"]]
        .merge(details, left_on="rec_groupId", right_on="groupId", how="left")
        .drop(columns=["groupId"])  # joined key
        [["rec_rank","similarity","rec_groupId"] + [c for c in keep if c != "groupId"]]
        .sort_values("rec_rank")
        .reset_index(drop=True)
)

view


,rec_rank,similarity,rec_groupId,name,name.1,audience,category,priceband,description
0,1,0.955854,579830,Kanallängd Malena,Linea,hemmet,"Kanallängder,Gardiner (linea)",Popular,Malena kanallängd är snygga längder i ett av våra mest populära mönster som finns i flera olika färgkombinationer.Material
1,2,0.951619,500355,Madrasskydd i 6 storlekar,Linea,hemmet,"Bädd,Bäddtillbehör,Bädd (linea)",Value,Madrasskydd i en tålig bomull- och polyesterblandning. Madrasskyddet är quiltat och formsytt samt hålls på plats med hjälp av resårband på kortsidorna. Finns i flera olika storlekar.Material
2,3,0.947816,521879,Innerkudde Rund,Linea,hemmet,"Innerkuddar,Bädd (linea)",Value,"Rund innerkudde med ett yttertyg av bomull och polyester, fyllning av polyester. Kudden är Oeko-Tex certifierad vilket är ett intyg på att den inte orsakar hälsoproblem. Storlek"
3,4,0.946706,520044,Kanallängd Stina,Linea,hemmet,"Kanallängder,Gardiner (linea)",Premium,Stilrena gardiner i ett garnfärgat rutmönster som gör gardinen lika fin på båda sidor. Finns i flera vackra färger.Material
4,5,0.944257,537138,Kanallängd Ingrid,Linea,hemmet,Kanallängder,Premium,Klassiska gardinlängder med ett garnfärgat rutmönster i vackra färgkombinationer .Då väven är garnfärgad blir längderna lika fin på båda sidor. Rutiga omtag 7x70cm medföljer.Material
5,6,0.943955,520039,Gardinkappa Stina,Linea,hemmet,"Kanalkappa,Gardiner (linea)",Value,Stilren gardinkappa i ett garnfärgat rutmönster som gör kappan lika fin på båda sidor. Finns i flera vackra färger.Material
6,7,0.942605,585060,Innertäcke Medium,Linea,hemmet,"Täcken,Bädd (linea)",Popular,"Mjukt quiltstickat innertäcke med ett elegant yttertyg av vävd kvalitet. Ett bandkant går längst med hela yttertyget. Fyllningen är gjord av silikoniserad polyester, hålfiber.Vikt"
7,8,0.942605,585061,Innertäcke Varm,Linea,hemmet,"Täcken,Bädd (linea)",Popular,"Mjukt quiltstickat innertäcke med ett elegant yttertyg av vävd kvalitet. Ett bandkant går längst med hela yttertyget. Fyllningen är gjord av silikoniserad polyester, hålfiber.Vikt"
8,9,0.942262,551100,Bordsduk Sara blommig,Linea,hemmet,Dukar,Value,Underbar duk med ett tryckt blommotiv i härliga pastellfärger. Ett enkelt sätt att göra din dukning lite extra fin.
9,10,0.940287,552175,Kanallängder garnfärgad ruta,Linea,hemmet,Kanallängder,Popular,Stilrena gardiner i ett garnfärgat rutmönster som gör gardinen lika fin på båda sidor. Finns i flera vackra färger.


In [71]:
# Visualize the recommendations for the target itself (i.e., show the row for the target)
cols = ["rec_rank", "similarity", "rec_groupId"] + [c for c in keep if c != "groupId"]
if "description" not in cols:
    cols.append("description")
view_target = (
    details.loc[details["groupId"] == target]
        .assign(rec_rank=1, similarity=1.0, rec_groupId=target)
        [cols]
        .reset_index(drop=True)
)
view_target


,rec_rank,similarity,rec_groupId,name,name.1,audience,category,priceband,description
0,1,1.0,190041,Innerkudde,Linea,hemmet,"Kuddar,Innerkuddar,Bädd (linea)",Value,Fyllnadskudde till kuddfodral med ett yttertyg av polyester/bomull och fyllning av polyesterfibrer. Innerkudden finns i flera olika storlekar för att matcha alla våra olika kuddfodral. Färg


In [72]:
out = (recs
       .sort_values(['src_groupId','rec_rank'])
       .groupby('src_groupId')['rec_groupId']
       .apply(list)
       .reset_index(name='recs'))

out.to_csv("../data/predictions/vector_similarity_recommendations.csv", index=False)


In [52]:
out

,src_groupId,recs
0,0044,"[870023, 263012, 261262, 263202, 261172, 261024, 262089, 262097, 263103, 263186]"
1,052743,"[520768, 522383, 569744, 569694, 581173, 528417, 582109, 562744, 582111, 522391]"
2,055522,"[170005, 273763, 261909, 261883, 261891, 261917, 273755, 890030, 210277, 207340]"
3,055573,"[055575, 055576, 319525, 200260, 587964, 262451, 260075, 970100, 970101, 340745]"
4,055575,"[055576, 055573, 319525, 200260, 587964, 262451, 260075, 340745, 970101, 970100]"
...,...,...
8357,AH2021,"[AH3021, AH2031, AH1021, 266965, 266940, 267005, 267013, 266981, 266973, 266361]"
8358,AH2031,"[AH2021, AH3021, AH1021, 266965, 266940, 267005, 267013, 266981, 266973, 266361]"
8359,AH3021,"[AH2021, AH2031, AH1021, 266965, 266940, 267005, 267013, 266981, 266973, 266361]"
8360,SOFIND,"[205443, 527499, 395713, 120020, 394550, 202820, 201202, 395700, 517029, 270140]"
